In [ ]:
import math
import pickle

import keras
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from keras.callbacks import Callback
from keras.layers import LSTM, Activation, Dense
from keras.models import Sequential
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
from numpy import concatenate
from pandas import DataFrame, concat, read_csv
from Queue import Queue
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

%matplotlib inline

In [ ]:
data_healthy = pickle.load(open("watsoniotp.healthy.pickle", "rb"))
data_broken = pickle.load(open("watsoniotp.broken.pickle", "rb"))

In [ ]:
data_healthy = data_healthy.reshape(3000, 3)
data_broken = data_broken.reshape(3000, 3)

In [ ]:
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [ ]:
data_healthy_scaled = scaleData(data_healthy)
data_broken_scaled = scaleData(data_broken)

In [ ]:
timesteps = 10
dim = 3
samples = 300
data_healthy_scaled_reshaped = data_healthy_scaled

In [ ]:
# design network

model = Sequential()
model.add(LSTM(50, input_shape=(timesteps, dim), return_sequences=False))
model.add(Dense(timesteps * dim))
model.compile(
    loss="mse",
    optimizer=keras.optimizers.SGD(
        lr=0.01, momentum=0.95, decay=5e-4, nesterov=True
    ),
)

## Now use Keras2DML importer

In [ ]:
from systemml.mllearn import Keras2DML

epochs = 50
batch_size = 50
max_iter = int(epochs * math.ceil(samples / batch_size))
model = Keras2DML(
    spark,
    model,
    input_shape=(timesteps, dim, 1),
    batch_size=batch_size,
    max_iter=max_iter,
    test_interval=0,
    display=10,
)
model.set(perform_one_hot_encoding=False)
model.set(debug=True)

In [ ]:
def train(data):
    data = data.reshape(samples, timesteps * dim)
    model.fit(
        data, data
    )  # epochs=50, batch_size=72, validation_data=(data, data), verbose=0, shuffle=False,callbacks=[LossHistory()])

In [ ]:
train(data_healthy_scaled)